In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.2 MB/s eta 0:00:00


In [ ]:
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [ ]:
# import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import sys
import csv
import gzip
import copy
import datetime
import pickle
from sklearn import metrics
from tabulate import tabulate

In [ ]:
seed_value = 42  # seed for reproducibility
random.seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.path.append('/content/drive/MyDrive/ctr/code/model')

In [ ]:
import dcn
import run_models

In [ ]:
BATCH_SIZE = 100000
SHUFFLE_BUFFER_SIZE = 10000
SHUFFLE_SEED = 42
DATASET_EPOCHS = 1
FITTING_EPOCHS = 50
LEARNING_RATE = 0.0001
DCN_PARALLEL = True
STR_COLUMNS = [
    'click', 'banner_pos',
    'site_id', 'site_domain', 'site_category',
    'app_id', 'app_domain', 'app_category',
    'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type',
    'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21',
    'day_of_week'
    ]
INT_COLUMNS = [
    'hour_of_day', 'rare count'
]
COLUMN_DEFAULTS = [tf.string] * (len(STR_COLUMNS) -1) + [tf.float32] + [tf.string] * 1 + [tf.float32]

In [ ]:
# column_defaults를 원본 CSV File의 Column 순서로 인식한다

In [ ]:
TRAIN_FILE = '/content/drive/MyDrive/ctr/avazu/processed/train/train_rare_count.csv'
train_batches = tf.data.experimental.make_csv_dataset(
    TRAIN_FILE,
    batch_size=BATCH_SIZE,
    select_columns= STR_COLUMNS + INT_COLUMNS,
    column_defaults=COLUMN_DEFAULTS,
    shuffle=True, shuffle_buffer_size=SHUFFLE_BUFFER_SIZE, shuffle_seed=SHUFFLE_SEED,
    num_epochs=DATASET_EPOCHS
)

In [ ]:
TEST_FILE = '/content/drive/MyDrive/ctr/avazu/processed/train/test_rare_count.csv'
test = tf.data.experimental.make_csv_dataset(
    TEST_FILE,
    batch_size=BATCH_SIZE,
    select_columns= STR_COLUMNS + INT_COLUMNS,
    column_defaults=COLUMN_DEFAULTS,
    num_epochs=DATASET_EPOCHS
)

In [ ]:
import pickle
with open('/content/drive/MyDrive/ctr/data/total_voca.p', 'rb') as f:
    total_voca = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/ctr/data/one_hot_encoding_voca.p', 'rb') as f:
    one_hot_encoding_voca = pickle.load(f)

In [ ]:
one_hot_encoding_voca['day_of_week'] = ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday',
       'Monday']

In [ ]:
FEATURE_ENGINEERING = {
    'one-hot encoding' : ['site_category', 'app_category', 'device_type', 'C1', 'C15', 'C16', 'C18', 'C21', 'day_of_week'],
    'threshold + embedding' : ['device_model', 'C14', 'C17', 'site_id', 'site_domain', 'app_id'],
    'top-n + one-hot encoding' : ['app_domain', 'C19', 'C20']
}

In [ ]:
THRESHOLD_VOCA = total_voca[(10,1000)]
TOP_N_VOCA = total_voca[(25, 500)]

In [ ]:
THRESHOLD_VOCA.keys()

dict_keys(['site_id', 'site_domain', 'device_model', 'app_id', 'app_domain', 'C14', 'C17', 'C19', 'C20'])

In [ ]:
vocabularies = {}

vocabularies['one-hot encoding'] = {}
for feature in FEATURE_ENGINEERING['one-hot encoding']:
    vocabularies['one-hot encoding'][feature] = one_hot_encoding_voca[feature]

vocabularies['threshold + embedding'] = {}
for feature in FEATURE_ENGINEERING['threshold + embedding']:
    vocabularies['threshold + embedding'][feature] = THRESHOLD_VOCA[feature]['over_threshold']

vocabularies['top-n + one-hot encoding'] = {}
for feature in FEATURE_ENGINEERING['top-n + one-hot encoding']:
    vocabularies['top-n + one-hot encoding'][feature] = TOP_N_VOCA[feature]['top_num']

In [ ]:
str_features = list(vocabularies['one-hot encoding'].keys()) + list(vocabularies['threshold + embedding'].keys()) + list(vocabularies['top-n + one-hot encoding'].keys())
int_features = ['hour_of_day']

In [ ]:
nodes = [32, 64, 128, 256, 512, 1024]
deep_layers = [1, 2, 3, 4, 5]
cross_layers = [1, 2, 3, 4, 5, 6]
result_dict = {}

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
for engineering_method, voca_set in vocabularies.items():
    for feature, voca in voca_set.items():
        vocabularies[engineering_method][feature] = list(map(str, voca))

In [ ]:
HISTORY_FILE_DIR = '/content/drive/MyDrive/ctr/data/feature_engineering.rare_count/'
HISTORY_FILE_NAME = 'fitting_history.V1.2'

In [ ]:
node, deep_layer, cross_layer = 1024, 4, 1
print("Node: {}, Deep Layer : {}, Cross Layer : {}".format(node, deep_layer, cross_layer))

fitting_history = run_models.run_models(
    dcn_parallel=DCN_PARALLEL,
    cross_layer_size=cross_layer,
    deep_layer_sizes=[node]*deep_layer,
    vocabularies=vocabularies,
    str_features=str_features,
    int_features=int_features,
    train=train_batches,
    test=test,
    learning_rate=LEARNING_RATE,
    epochs=FITTING_EPOCHS,
    history_file_dir=HISTORY_FILE_DIR,
    history_file_name=HISTORY_FILE_NAME
    )

Node: 1024, Deep Layer : 4, Cross Layer : 1
363/363 ━━━━━━━━━━━━━━━━━━━━ 1174s 3s/step - AUC: 0.6106 - LogLoss: 0.6034 - loss: 0.5516 - regularization_loss: 0.0000e+00 - total_loss: 0.5516
43/43 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - AUC: 0.5415 - LogLoss: 2.0947 - loss: 2.5719 - regularization_loss: 0.0000e+00 - total_loss: 2.5719
1th Epoch
363/363 ━━━━━━━━━━━━━━━━━━━━ 1148s 3s/step - AUC: 0.6312 - LogLoss: 0.6067 - loss: 0.5058 - regularization_loss: 0.0000e+00 - total_loss: 0.5058
43/43 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - AUC: 0.6818 - LogLoss: 0.4284 - loss: 0.4721 - regularization_loss: 0.0000e+00 - total_loss: 0.4721
363/363 ━━━━━━━━━━━━━━━━━━━━ 1159s 3s/step - AUC: 0.6981 - LogLoss: 0.4340 - loss: 0.4348 - regularization_loss: 0.0000e+00 - total_loss: 0.4348
43/43 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - AUC: 0.6849 - LogLoss: 0.4281 - loss: 0.4855 - regularization_loss: 0.0000e+00 - total_loss: 0.4855
363/363 ━━━━━━━━━━━━━━━━━━━━ 1124s 3s/step - AUC: 0.6896 - LogLoss: 0.4384 - loss: 0.

In [ ]:
import pickle
with open(HISTORY_FILE_PATH, 'wb') as f:
    pickle.dump(fitting_history, f)